# 1. EntropyMeasure

---

## 1.1 Hadamard Test - multiple experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_hadamard = EntropyMeasure(method="hadamard")

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_hadamard.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_hadamard.add(GHZ(i), f"ghz_{i}")
    experiment_hadamard.add(make_neel_circuit(i), f"neel_{i}")

experiment_hadamard.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a8f818e7590>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x7a8f818e7080>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7a8f819a53a0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a8f81710710>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x7a8f817107a0>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7a8f81710980>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a8f81710b30>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7a8f81710bc0>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7a903d6c7d10>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7a8ff519c230>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x7a8f81711160>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.hadamard_test.arguments import EntropyMeasureHadamardMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureHadamardMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""
    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    degree: Optional[Union[int, tuple[int, int]]]
    """The degree range."""
```


In [4]:
config_list: list[EntropyMeasureHadamardMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "degree": i // 2,
        "tags": (wave_names, f"size_{i}", f"system_range_{i//2}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_hadamard.multiOutput(
    config_list,
    summoner_name="qurrent.hadamard_test",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurrent.hadamard_test.001", at location "qurrent.hadamard_test.001"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for f4e034cc-b08a-41d8-933a-46f5ef693087


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.hadamard_test.001/qurryinfo.json...
| Exporting qurrent.hadamard_test.001/qurryinfo.json done.


'f4e034cc-b08a-41d8-933a-46f5ef693087'

In [6]:
experiment_hadamard.multimanagers[multi_exps1]

<MultiManager(id="f4e034cc-b08a-41d8-933a-46f5ef693087",
  name="qurrent.hadamard_test.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-21 22:37:51',},
  exps_num=180)>

### d. Run post-processing at once


In [7]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'f4e034cc-b08a-41d8-933a-46f5ef693087'

In [8]:
print("| Available results:")
for k, v in (
    experiment_hadamard.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_10', 'system_range_5') with length 10
| - ('ghz', 'size_10', 'system_range_5') with length 10
| - ('neel', 'size_10', 'system_range_5') with length 10
| - ('trivial_paramagnet', 'size_12', 'system_range_6') with length 10
| - ('ghz', 'size_12', 'system_range_6') with length 10
| - ('neel', 'size_12', 'system_range_6') with length 10
| - ('trivial_paramagnet', 'size_2', 'system_range_1') with length 10
| - ('ghz', 'size_2', 'system_range_1') with length 10
| - ('neel', 'size_2', 'system_range_1') with length 10
| - ('trivial_paramagnet', 'size_4', 'system_range_2') with length 10
| - ('ghz', 'size_4', 'system_range_2') with length 10
| - ('neel', 'size_4', 'system_range_2') with length 10
| - ('trivial_paramagnet', 'size_6', 'system_range_3') with length 10
| - ('ghz', 'size_6', 'system_range_3') with length 10
| - ('neel', 'size_6', 'system_range_3') with length 10
| - ('trivial_paramagnet', 'size_8', 'system_range_4') with length 1

In [9]:
experiment_hadamard.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10", "system_range_5")
][:2]

[{'purity': 1.0,
  'entropy': np.float64(-0.0),
  'input': {},
  'header': {'serial': 0,
   'datetime': '2025-05-21 22:37:58',
   'summoner': None,
   'log': {}}},
 {'purity': 1.0,
  'entropy': np.float64(-0.0),
  'input': {},
  'header': {'serial': 0,
   'datetime': '2025-05-21 22:37:58',
   'summoner': None,
   'log': {}}}]

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(
    experiment_hadamard.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={k: idx < 3 for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'f4e034cc-b08a-41d8-933a-46f5ef693087'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_hadamard.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={k: {} for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for f4e034cc-b08a-41d8-933a-46f5ef693087


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.hadamard_test.001/qurryinfo.json...
| Exporting qurrent.hadamard_test.001/qurryinfo.json done.


'f4e034cc-b08a-41d8-933a-46f5ef693087'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_hadamard.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_hadamard.multiRead(
    save_location=".",
    summoner_name="qurrent.hadamard_test.001",
)

| Retrieve qurrent.hadamard_test.001...
| at: qurrent.hadamard_test.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

In [16]:
expkeys_of_multi_exps1

['17a59e83-2db7-4d43-9022-d74b10ed5b93',
 '50ecbbcf-af4d-4033-9e95-98d7f19eecef',
 '03858d65-b4fe-4ce0-bbd2-23dc7d76244b',
 '4e4cf513-a5eb-4701-b5b2-128f0f2b1e69',
 '458ddf6a-3ccf-40da-9707-580a137c13ee',
 '65f51ebc-84e1-47ba-9a41-ab3a37de13d9',
 '0861a354-8418-4831-a572-a15b73fd6149',
 'b15e7379-ba66-4d8f-b996-27e7807443ff',
 '5da36fba-8054-47d9-87e8-30e2ff193e2a',
 '7749e118-c98b-40b1-addb-693a5cc0d7e4',
 '308c99c9-9af9-4919-a709-8aaa07755fe2',
 '4aef5a93-dab6-42a6-bf71-703e092eb384',
 'd56f041c-d0ab-40fc-a42d-57484e73e81f',
 '65099f19-35a4-4516-b087-9852372e2a5a',
 'a37a490c-c12d-4eea-b02f-f8435b546dc5',
 'd7dc313d-82b0-491e-879c-f466cf208928',
 '25ec9908-0ee5-49e4-a2a3-69a8dbd36c81',
 '13ffd710-8caf-4812-a87a-30633afa1a04',
 'cebfc306-29f0-433b-ba91-af2a790efc8b',
 'b566da9e-3c76-4a74-a687-3b656aa97dea',
 'f584c4a7-12a3-4682-9c5a-c9c184fadafd',
 '07a0b63f-8dd5-468a-8ccd-dcf1174a6fb1',
 '82e10737-46fb-4f72-a588-79925faaa8d5',
 '8dd51005-7949-428f-98dc-8c392adece3d',
 '17e22559-7484-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET
AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev1
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  